1. Частеречный анализ фразы с помощью Mystem

In [1]:
from pymystem3 import Mystem

text = 'Не бойся идти медленно, бойся стоять на месте'

In [2]:
# Создаем экземпляр Mystem
mystem = Mystem()

In [3]:
pos = mystem.analyze(text)
for analysis in pos:
    if 'analysis' in analysis and analysis['analysis']:
        # Извлекаем часть речи из грамматической информации
        gr = analysis['analysis'][0]['gr']
        # Первый элемент в gr обычно содержит часть речи
        pos_tag = gr.split('=')[0].split(',')[0]
        print(f"{analysis['text']} - {pos_tag}")
    else:
        print(f"{analysis['text']} - (без анализа)")

Не - PART
  - (без анализа)
бойся - V
  - (без анализа)
идти - V
  - (без анализа)
медленно - ADV
,  - (без анализа)
бойся - V
  - (без анализа)
стоять - V
  - (без анализа)
на - PR
  - (без анализа)
месте - S

 - (без анализа)


2. Векторное представление с помощью TF-IDF с триграммами и леммами

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [7]:
# Лемматизация текста
lemmas_mystem = mystem.lemmatize(text)
lemmatized_text = "".join(lemmas_mystem).strip()
print(f"\nЛемматизированный текст: {lemmatized_text}")


Лемматизированный текст: не бояться идти медленно, бояться стоять на место


In [8]:
# Создаем TF-IDF вектор с триграммами
tfidf_trigram = TfidfVectorizer(ngram_range=(3, 3))
text_collection = [lemmatized_text]

In [9]:
# Преобразуем данные
transformed_trigram = tfidf_trigram.fit_transform(text_collection)

In [10]:
# Создаем таблицу данных с TF-IDF значениями
df_trigram = pd.DataFrame(transformed_trigram[0].T.todense(),
                         index=tfidf_trigram.get_feature_names_out(), 
                         columns=["TF-IDF"])
df_trigram = df_trigram.sort_values('TF-IDF', ascending=False)

print("\nTF-IDF вектор с триграммами (первые 10):")
print(df_trigram.head(10))


TF-IDF вектор с триграммами (первые 10):
                           TF-IDF
бояться идти медленно    0.408248
бояться стоять на        0.408248
идти медленно бояться    0.408248
медленно бояться стоять  0.408248
не бояться идти          0.408248
стоять на место          0.408248


3. Обучение модели skip-gram и вычисление вектора слова «Бойся»

In [11]:
import gensim
from gensim.models import Word2Vec

In [12]:
# Подготовка данных для Word2Vec
sentences = [text.lower().split()]

In [14]:
# Обучаем модель skip-gram (sg=1) с 50 нейронами в скрытом слое
skipgram_model = Word2Vec(sentences, 
                         min_count=1, 
                         vector_size=50, 
                         workers=4, 
                         window=2, 
                         sg=1)  # sg=1 для skip-gram

print(f"\nМодель Skip-gram:")
print(skipgram_model)


Модель Skip-gram:
Word2Vec<vocab=7, vector_size=50, alpha=0.025>


In [23]:
# Вычисляем вектор слова «Бойся» (в нижнем регистре, так как модель обучена на lowercase)
try:
    vector_bojsya = skipgram_model.wv['бойся']
    print(f"\nВектор слова 'бойся' (размерность {len(vector_bojsya)}):")
    print(vector_bojsya)
    
    # Также покажем похожие слова
    similar_words = skipgram_model.wv.most_similar('бойся', topn=3)
    print(f"\nСлова, наиболее похожие на 'бойся':")
    for word, similarity in similar_words:
        print(f"  {word}: {similarity:.4f}")
        
except KeyError:
    print("Слово 'бойся' не найдено в словаре модели")


Вектор слова 'бойся' (размерность 50):
[-1.07170537e-03  4.72441199e-04  1.02078691e-02  1.80197861e-02
 -1.86081287e-02 -1.42342113e-02  1.29182357e-02  1.79479849e-02
 -1.00312335e-02 -7.52825011e-03  1.47622544e-02 -3.06720077e-03
 -9.07347351e-03  1.31098470e-02 -9.72088240e-03 -3.63255292e-03
  5.75367268e-03  1.98333175e-03 -1.65713690e-02 -1.88992545e-02
  1.46240527e-02  1.01404227e-02  1.35165276e-02  1.52677775e-03
  1.27022704e-02 -6.81129051e-03 -1.89369719e-03  1.15380771e-02
 -1.50441136e-02 -7.87189696e-03 -1.50241815e-02 -1.86050800e-03
  1.90770905e-02 -1.46401254e-02 -4.66730166e-03 -3.87595571e-03
  1.61554441e-02 -1.18621765e-02  9.06468704e-05 -9.50885657e-03
 -1.92086790e-02  1.00153033e-02 -1.75208114e-02 -8.78431834e-03
 -7.11813336e-05 -5.92919998e-04 -1.53227011e-02  1.92316137e-02
  9.96475667e-03  1.84669942e-02]

Слова, наиболее похожие на 'бойся':
  на: 0.1267
  месте: 0.0423
  идти: 0.0124


In [21]:
# Дополнительная информация о модели
print(f"\nСлова в словаре модели: {list(skipgram_model.wv.key_to_index.keys())}")
print(f"Размер словаря: {len(skipgram_model.wv.key_to_index)}")


Слова в словаре модели: ['бойся', 'месте', 'на', 'стоять', 'медленно,', 'идти', 'не']
Размер словаря: 7
